In [2]:
import pandas
import math

table_16_4 = pandas.read_excel("16.4.xlsx")
table_16_5 = pandas.read_excel("16.5.xlsx")
table_16_6 = pandas.read_excel("16.6.xlsx")
table_16_7 = pandas.read_excel("16.7.xlsx")
table_16_8 = pandas.read_excel("16.8.xlsx")
table_16_9 = pandas.read_excel("16.9.xlsx")
table_16_10 = pandas.read_excel("16.10.xlsx")
table_16_11 = pandas.read_excel("16.11.xlsx")
table_16_12 = pandas.read_excel("16.12.xlsx")
table_16_13 = pandas.read_excel("16.13.xlsx")
table_16_15 = pandas.read_excel("16.15.xlsx")
table_16_16 = pandas.read_excel("16.16.xlsx")
table_16_17 = pandas.read_excel("16.17.xlsx")

def interpolation(x1, x2, y1, y2, x):
  return float(y1) + (float(y2) - float(y1)) / (float(x2) - float(x1)) * (float(x) - float(x1))

def calculateFLS(laneWidth, shoulderWidth):
  row = 0
  col = 0
  for id, item in enumerate(table_16_5.iloc[:, 0]):
    if type(item) == str:
      laneWidthRange = item.split(",")
      if len(laneWidthRange) == 2:
        laneWidthFrom = laneWidthRange[0]
        laneWidthUpto = laneWidthRange[1]
        if laneWidthFrom and laneWidthUpto:
          if laneWidthFrom <= laneWidth < laneWidthUpto:
            row = id
        elif laneWidthFrom:
          if laneWidthFrom <= laneWidth:
            row = id
  for id, item in enumerate(table_16_5.iloc[0]):
    if type(item) == str:
      shoulderWidthRange = item.split(",")
      if len(shoulderWidthRange) == 2:
        shoulderWidthFrom = shoulderWidthRange[0]
        shoulderWidthUpto = shoulderWidthRange[1]
        if shoulderWidthFrom and shoulderWidthUpto:
          if shoulderWidthFrom <= shoulderWidth < shoulderWidthUpto:
            col = id
        elif shoulderWidthFrom:
          if shoulderWidthFrom <= shoulderWidth:
            col = id
  return table_16_5.iloc[row, col]

def calculateFA(accessPoints):
  prevId = 0
  for id, item in enumerate(table_16_6["access_point_per_mile"]):
    if float(item) == float(accessPoints):
      return table_16_6["reduction_in_ffs"][id]
    elif float(item) < float(accessPoints):
      prevId = id
    else:
      nextId = id
      x1 = table_16_6["access_point_per_mile"][prevId]
      x2 = table_16_6["access_point_per_mile"][nextId]
      y1 = table_16_6["reduction_in_ffs"][prevId]
      y2 = table_16_6["reduction_in_ffs"][nextId]
      return round(interpolation(x1, x2, y1, y2, accessPoints), 2)

def calculateFG(gradeCalculationType, v, gradeType = None, grade = None, gradeLength = None):
  fGATS = None
  fGPTSF = None
  # fG for both ATS and PTSF using table 16.7
  if gradeCalculationType == "1":
    if gradeType == "1":
      fGATS = 1
      fGPTSF = 1
    else:
      prevId = 0
      for id, item in enumerate(table_16_7["v"]):
        if item == v:
          fGATS = table_16_7["rolling_terrain_ats"][id]
          fGPTSF = table_16_7["rolling_terrain_ptsf"][id]
          break
        elif float(item) < float(v):
          prevId = id
          if id == len(table_16_7["v"]) - 1:
            fGATS = table_16_7["rolling_terrain_ats"][id]
            fGPTSF = table_16_7["rolling_terrain_ptsf"][id]
        else:
          if id == 0:
            fGATS = table_16_7["rolling_terrain_ats"][id]
            fGPTSF = table_16_7["rolling_terrain_ptsf"][id]
            break
          nextId = id
          x1ATS = table_16_7["v"][prevId]
          x2ATS = table_16_7["v"][nextId]
          y1ATS = table_16_7["rolling_terrain_ats"][prevId]
          y2ATS = table_16_7["rolling_terrain_ats"][nextId]
          fGATS = round(interpolation(x1ATS, x2ATS, y1ATS, y2ATS, v), 2)

          x1PTSF = table_16_7["v"][prevId]
          x2PTSF = table_16_7["v"][nextId]
          y1PTSF = table_16_7["rolling_terrain_ptsf"][prevId]
          y2PTSF = table_16_7["rolling_terrain_ptsf"][nextId]
          fGPTSF = round(interpolation(x1PTSF, x2PTSF, y1PTSF, y2PTSF, v), 2)
          break

  # fG for ATS using table 16.8 and for PTSF using table 16.9
  else:
    col = None
    prevCol = None
    nextCol = None
    row = None
    prevRow = None
    nextRow = None
    prevId = 0

    # col calculation for fG
    for id, item in enumerate(table_16_8.iloc[0]):
      if type(item) != str:
        if item == v:
          col = id
          break
        elif float(item) < float(v):
          prevId = id
          if id == len(table_16_8.iloc[0]) - 1:
            col = id
        else:
          if id == 2:
            col = id
            break
          prevCol = prevId
          nextCol = id
          break

    # row calculation for fG for ATS using table 16.8
    for id, item in enumerate(table_16_8.iloc[:, 0]):
      if type(item) == str:
        gradeRange = item.split(",")
        if len(gradeRange) == 2:
          gradeFrom = gradeRange[0]
          gradeUpto = gradeRange[1]
          trueRange = False
          if gradeFrom and gradeUpto:
            if float(gradeFrom) <= float(grade) < float(gradeUpto):
              trueRange = True
          elif gradeFrom:
            if float(gradeFrom) <= float(grade):
              trueRange = True
          if trueRange:
            gradeLengthItem = table_16_8.iloc[:, 1][id]
            if float(gradeLengthItem) == float(gradeLength):
              row = id
              break
            elif float(gradeLengthItem) < float(gradeLength):
              if float(gradeLengthItem) == 4:
                row = id
                break
              prevRow = id
            else:
              if float(gradeLengthItem) == 0.25:
                row = id
                break
              nextRow = id
              break
    if row and col:
      fGATS = table_16_8.iloc[row, col]
    elif row and prevCol and nextCol:
      x1 = table_16_8.iloc[0][prevCol]
      x2 = table_16_8.iloc[0][nextCol]
      y1 = table_16_8.iloc[row][prevCol]
      y2 = table_16_8.iloc[row][nextCol]
      fGATS = round(interpolation(x1, x2, y1, y2, v), 2)
    elif prevRow and nextRow and col:
      x1 = table_16_8.iloc[prevRow][1]
      x2 = table_16_8.iloc[nextRow][1]
      y1 = table_16_8.iloc[prevRow][col]
      y2 = table_16_8.iloc[nextRow][col]
      fGATS = round(interpolation(x1, x2, y1, y2, gradeLength), 2)
    else:
      x1 = table_16_8.iloc[0][prevCol]
      x2 = table_16_8.iloc[0][nextCol]
      y1 = table_16_8.iloc[prevRow][prevCol]
      y2 = table_16_8.iloc[prevRow][nextCol]
      Y1 = interpolation(x1, x2, y1, y2, v)

      x1 = table_16_8.iloc[0][prevCol]
      x2 = table_16_8.iloc[0][nextCol]
      y1 = table_16_8.iloc[nextRow][prevCol]
      y2 = table_16_8.iloc[nextRow][nextCol]
      Y2 = interpolation(x1, x2, y1, y2, v)

      x1 = table_16_8.iloc[prevRow][1]
      x2 = table_16_8.iloc[nextRow][1]
      fGATS = round(interpolation(x1, x2, Y1, Y2, gradeLength), 2)

    # row calculation for fG for PTSF using table 16.9
    for id, item in enumerate(table_16_9.iloc[:, 0]):
      if type(item) == str:
        gradeRange = item.split(",")
        if len(gradeRange) == 2:
          gradeFrom = gradeRange[0]
          gradeUpto = gradeRange[1]
          trueRange = False
          if gradeFrom and gradeUpto:
            if float(gradeFrom) <= float(grade) < float(gradeUpto):
              trueRange = True
          elif gradeFrom:
            if float(gradeFrom) <= float(grade):
              trueRange = True
          if trueRange:
            gradeLengthItem = table_16_9.iloc[:, 1][id]
            if float(gradeLengthItem) == float(gradeLength):
              row = id
              break
            elif float(gradeLengthItem) < float(gradeLength):
              prevRow = id
              if float(gradeLengthItem) == 4 or float(gradeLengthItem) == 0:
                row = id
            else:
              if float(gradeLengthItem) == 0.25 or float(gradeLengthItem) == 0:
                row = id
                break
              nextRow = id
              break
    if row and col:
      fGPTSF = table_16_9.iloc[row, col]
    elif row and prevCol and nextCol:
      x1 = table_16_9.iloc[0][prevCol]
      x2 = table_16_9.iloc[0][nextCol]
      y1 = table_16_9.iloc[row][prevCol]
      y2 = table_16_9.iloc[row][nextCol]
      fGPTSF = round(interpolation(x1, x2, y1, y2, v), 2)
    elif prevRow and nextRow and col:
      x1 = table_16_9.iloc[prevRow][1]
      x2 = table_16_9.iloc[nextRow][1]
      y1 = table_16_9.iloc[prevRow][col]
      y2 = table_16_9.iloc[nextRow][col]
      fGPTSF = round(interpolation(x1, x2, y1, y2, gradeLength), 2)
    else:
      x1 = table_16_9.iloc[0][prevCol]
      x2 = table_16_9.iloc[0][nextCol]
      y1 = table_16_9.iloc[prevRow][prevCol]
      y2 = table_16_9.iloc[prevRow][nextCol]
      Y1 = interpolation(x1, x2, y1, y2, v)

      x1 = table_16_9.iloc[0][prevCol]
      x2 = table_16_9.iloc[0][nextCol]
      y1 = table_16_9.iloc[nextRow][prevCol]
      y2 = table_16_9.iloc[nextRow][nextCol]
      Y2 = interpolation(x1, x2, y1, y2, v)

      x1 = table_16_9.iloc[prevRow][1]
      x2 = table_16_9.iloc[nextRow][1]
      fGPTSF = round(interpolation(x1, x2, Y1, Y2, gradeLength), 2)

  return fGATS, fGPTSF

def calculateFHV(gradeCalculationType, v, PT, PR, gradeType = None, grade = None, gradeLength = None):
  fHVATS = None
  fHVPTSF = None
  ETATS = None
  ETPTSF = None
  ERATS = None
  ERPTSF = None
  # ET and ER for both ATS and PTSF using table 16.10
  if gradeCalculationType == "1":
    if gradeType == "1":
      prevId = 0
      for id, item in enumerate(table_16_10["v"]):
        if item == v:
          ETATS = table_16_10["level_terrain_ats"][id]
          ETPTSF = table_16_10["level_terrain_ptsf"][id]
          break
        elif float(item) < float(v):
          prevId = id
          if id == len(table_16_10["v"]) - 1:
            ETATS = table_16_10["level_terrain_ats"][id]
            ETPTSF = table_16_10["level_terrain_ptsf"][id]
        else:
          if id == 0:
            ETATS = table_16_10["level_terrain_ats"][id]
            ETPTSF = table_16_10["level_terrain_ptsf"][id]
            break
          nextId = id
          x1ATS = table_16_10["v"][prevId]
          x2ATS = table_16_10["v"][nextId]
          y1ATS = table_16_10["level_terrain_ats"][prevId]
          y2ATS = table_16_10["level_terrain_ats"][nextId]
          ETATS = round(interpolation(x1ATS, x2ATS, y1ATS, y2ATS, v), 2)

          x1PTSF = table_16_10["v"][prevId]
          x2PTSF = table_16_10["v"][nextId]
          y1PTSF = table_16_10["level_terrain_ptsf"][prevId]
          y2PTSF = table_16_10["level_terrain_ptsf"][nextId]
          ETPTSF = round(interpolation(x1PTSF, x2PTSF, y1PTSF, y2PTSF, v), 2)
          break
      ERATS = table_16_10["level_terrain_ats"][9]
      ERPTSF = table_16_10["level_terrain_ptsf"][9]
    else:
      prevId = 0
      for id, item in enumerate(table_16_10["v"]):
        if item == v:
          ETATS = table_16_10["rolling_terrain_ats"][id]
          ETPTSF = table_16_10["rolling_terrain_ptsf"][id]
          break
        elif float(item) < float(v):
          prevId = id
          if id == len(table_16_10["v"]) - 1:
            ETATS = table_16_10["rolling_terrain_ats"][id]
            ETPTSF = table_16_10["rolling_terrain_ptsf"][id]
        else:
          if id == 0:
            ETATS = table_16_10["rolling_terrain_ats"][id]
            ETPTSF = table_16_10["rolling_terrain_ptsf"][id]
            break
          nextId = id
          x1ATS = table_16_10["v"][prevId]
          x2ATS = table_16_10["v"][nextId]
          y1ATS = table_16_10["rolling_terrain_ats"][prevId]
          y2ATS = table_16_10["rolling_terrain_ats"][nextId]
          ETATS = round(interpolation(x1ATS, x2ATS, y1ATS, y2ATS, v), 2)

          x1PTSF = table_16_10["v"][prevId]
          x2PTSF = table_16_10["v"][nextId]
          y1PTSF = table_16_10["rolling_terrain_ptsf"][prevId]
          y2PTSF = table_16_10["rolling_terrain_ptsf"][nextId]
          ETPTSF = round(interpolation(x1PTSF, x2PTSF, y1PTSF, y2PTSF, v), 2)
          break

      ERATS = table_16_10["rolling_terrain_ats"][9]
      ERPTSF = table_16_10["rolling_terrain_ptsf"][9]

  # ET and ER for ATS and PTSF using tables 16.11, 16.12, 16.13
  else:
    col = None
    prevCol = None
    nextCol = None
    prevId = 0

    # pre calculation for ET and ER
    for id, item in enumerate(table_16_11.iloc[0]):
      if type(item) != str:
        if item == v:
          col = id
          break
        elif float(item) < float(v):
          prevId = id
          if id == len(table_16_11.iloc[0]) - 1:
            col = id
        else:
          if id == 2:
            col = id
            break
          prevCol = prevId
          nextCol = id
          break

    row = None
    prevRow = None
    nextRow = None
    # ET for ATS using table 16.11
    for id, item in enumerate(table_16_11.iloc[:, 0]):
      if type(item) == str:
        gradeRange = item.split(",")
        if len(gradeRange) == 2:
          gradeFrom = gradeRange[0]
          gradeUpto = gradeRange[1]
          trueRange = False
          if gradeFrom and gradeUpto:
            if float(gradeFrom) <= float(grade) < float(gradeUpto):
              trueRange = True
          elif gradeFrom:
            if float(gradeFrom) <= float(grade):
              trueRange = True
          if trueRange:
            gradeLengthItem = table_16_11.iloc[:, 1][id]
            if float(gradeLengthItem) == float(gradeLength):
              row = id
              break
            elif float(gradeLengthItem) < float(gradeLength):
              if float(gradeLengthItem) == 4:
                row = id
                break
              prevRow = id
            else:
              if float(gradeLengthItem) == 0.25:
                row = id
                break
              nextRow = id
              break
    if row and col:
      ETATS = table_16_11.iloc[row, col]
    elif row and prevCol and nextCol:
      x1 = table_16_11.iloc[0][prevCol]
      x2 = table_16_11.iloc[0][nextCol]
      y1 = table_16_11.iloc[row][prevCol]
      y2 = table_16_11.iloc[row][nextCol]
      ETATS = round(interpolation(x1, x2, y1, y2, v), 2)
    elif prevRow and nextRow and col:
      x1 = table_16_11.iloc[prevRow][1]
      x2 = table_16_11.iloc[nextRow][1]
      y1 = table_16_11.iloc[prevRow][col]
      y2 = table_16_11.iloc[nextRow][col]
      ETATS = round(interpolation(x1, x2, y1, y2, gradeLength), 2)
    else:
      x1 = table_16_11.iloc[0][prevCol]
      x2 = table_16_11.iloc[0][nextCol]
      y1 = table_16_11.iloc[prevRow][prevCol]
      y2 = table_16_11.iloc[prevRow][nextCol]
      Y1 = interpolation(x1, x2, y1, y2, v)

      x1 = table_16_11.iloc[0][prevCol]
      x2 = table_16_11.iloc[0][nextCol]
      y1 = table_16_11.iloc[nextRow][prevCol]
      y2 = table_16_11.iloc[nextRow][nextCol]
      Y2 = interpolation(x1, x2, y1, y2, v)

      x1 = table_16_11.iloc[prevRow][1]
      x2 = table_16_11.iloc[nextRow][1]
      ETATS = round(interpolation(x1, x2, Y1, Y2, gradeLength), 2)

    # ER for ATS using table 16.12
    for id, item in enumerate(table_16_12.iloc[:, 0]):
      if type(item) == str:
        gradeRange = item.split(",")
        if len(gradeRange) == 2:
          gradeFrom = gradeRange[0]
          gradeUpto = gradeRange[1]
          trueRange = False
          if gradeFrom and gradeUpto:
            if float(gradeFrom) <= float(grade) < float(gradeUpto):
              trueRange = True
          elif gradeFrom:
            if float(gradeFrom) <= float(grade):
              trueRange = True
          if trueRange:
            gradeLengthItem = table_16_12.iloc[:, 1][id]
            if type(gradeLengthItem) == str:
              gradeLengthRange = gradeLengthItem.split(",")
              if len(gradeLengthRange) == 2:
                gradeLengthFrom = gradeLengthRange[0]
                gradeLengthUpto = gradeLengthRange[1]
                if gradeLengthFrom and gradeLengthUpto:
                  if float(gradeLengthFrom) <= float(gradeLength) < float(gradeLengthUpto):
                    row = id
                elif gradeLengthFrom:
                  if float(gradeLengthFrom) <= float(gradeLength):
                    row = id
                elif gradeLengthUpto:
                  if float(gradeLengthUpto) > float(gradeLength):
                    row = id

    if row and col:
      ERATS = table_16_12.iloc[row, col]
    else:
      x1 = table_16_12.iloc[0, prevCol]
      x2 = table_16_12.iloc[0, nextCol]
      y1 = table_16_12.iloc[row, prevCol]
      y2 = table_16_12.iloc[row, nextCol]
      ERATS = round(interpolation(x1, x2, y1, y2, v), 2)

    row = None
    prevRow = None
    nextRow = None
    # ET for PTSF using table 16.13
    for id, item in enumerate(table_16_13.iloc[:, 0]):
      if type(item) == str:
        gradeRange = item.split(",")
        if len(gradeRange) == 2:
          gradeFrom = gradeRange[0]
          gradeUpto = gradeRange[1]
          trueRange = False
          if gradeFrom and gradeUpto:
            if float(gradeFrom) <= float(grade) < float(gradeUpto):
              trueRange = True
          elif gradeFrom:
            if float(gradeFrom) <= float(grade):
              trueRange = True
          if trueRange:
            gradeLengthItem = table_16_13.iloc[:, 1][id]
            if float(gradeLengthItem) == float(gradeLength):
              row = id
              break
            elif float(gradeLengthItem) < float(gradeLength):
              if float(gradeLengthItem) == 4:
                row = id
                break
              prevRow = id
            else:
              if float(gradeLengthItem) == 0.5:
                row = id
                break
              nextRow = id
              break

    if row and col:
      ETPTSF = table_16_13.iloc[row, col]
    elif row and prevCol and nextCol:
      x1 = table_16_13.iloc[0][prevCol]
      x2 = table_16_13.iloc[0][nextCol]
      y1 = table_16_13.iloc[row][prevCol]
      y2 = table_16_13.iloc[row][nextCol]
      ETPTSF = round(interpolation(x1, x2, y1, y2, v), 2)
    elif prevRow and nextRow and col:
      x1 = table_16_13.iloc[prevRow][1]
      x2 = table_16_13.iloc[nextRow][1]
      y1 = table_16_13.iloc[prevRow][col]
      y2 = table_16_13.iloc[nextRow][col]
      ETPTSF = round(interpolation(x1, x2, y1, y2, gradeLength), 2)
    else:
      x1 = table_16_13.iloc[0][prevCol]
      x2 = table_16_13.iloc[0][nextCol]
      y1 = table_16_13.iloc[prevRow][prevCol]
      y2 = table_16_13.iloc[prevRow][nextCol]
      Y1 = interpolation(x1, x2, y1, y2, v)

      x1 = table_16_13.iloc[0][prevCol]
      x2 = table_16_13.iloc[0][nextCol]
      y1 = table_16_13.iloc[nextRow][prevCol]
      y2 = table_16_13.iloc[nextRow][nextCol]
      Y2 = interpolation(x1, x2, y1, y2, v)

      x1 = table_16_13.iloc[prevRow][1]
      x2 = table_16_13.iloc[nextRow][1]
      ETPTSF = round(interpolation(x1, x2, Y1, Y2, gradeLength), 2)

    # ER for PTSF
    ERPTSF = 1.0

  fHVATS = round(1 / (1 + float(PT) / 100 * (ETATS - 1) + float(PR) / 100 * (ERATS - 1)), 3)
  fHVPTSF = round(1 / (1 + float(PT) / 100 * (ETPTSF - 1) + float(PR) / 100 * (ERPTSF - 1)), 3)

  return fHVATS, fHVPTSF

def calculatefnp(FFS, v0, NPZ, dir1, v1, v2):
  fnpATS = None
  fnpPTSF = None
  prevId = 0
  col = None
  prevCol = None
  nextCol = None
  # pre calculation for fnp using table 16.15
  for id, item in enumerate(table_16_15.iloc[0]):
    if type(item) != str:
      if float(item) == float(NPZ):
        col = id
        break
      elif float(item) < float(NPZ):
        prevId = id
        if id == len(table_16_15.iloc[0]) - 1:
          col = id
      else:
        if id == 2:
          col = id
          break
        prevCol = prevId
        nextCol = id
        break

  prevId = 0
  row = None
  row11 = None
  row12 = None
  row21 = None
  row22 = None
  # fnp for ATS using table 16.15
  for id, item in enumerate(table_16_15.iloc[:, 0]):
    if type(item) != str:
      vItem = table_16_15.iloc[:, 1][id]
      if float(item) == float(FFS):
        if float(vItem) == float(v0):
          row = id
          row11 = None
          row12 = None
          row21 = None
          row22 = None
          break
        elif float(vItem) < float(v0):
          if float(vItem) == 1600:
            row = id
            continue
          prevId = id
        else:
          if float(vItem) == 100:
            row = id
            continue
          row11 = None
          row12 = None
          row21 = prevId
          row22 = id
          break
      elif float(item) > float(FFS):
        if float(vItem) == float(v0):
          prevId = id
        elif float(vItem) < float(v0):
          if float(vItem) == 1600:
            row11 = id
            continue
          prevId = id
          row12 = None
        elif not row12:
          if float(vItem) == 100:
            row11 = id
            continue
          row11 = prevId
          row12 = id
      else:
        if float(vItem) == float(v0):
          row12 = id
          break
        elif float(vItem) < float(v0):
          if float(vItem) == 1600:
            row12 = id
            break
          prevId = id
        elif not row22:
          if float(vItem) == 100:
            row12 = id
            break
          row21 = prevId
          row22 = id
  if row11 and not row12:
    row = row11
    row11 = None
  elif row12 and not row11:
    row = row21
    row12 = None

  if col and row:
    fnpATS = table_16_15.iloc[row, col]
  elif col and row11 and row12 and row21 and row22:
    x1 = table_16_15.iloc[row11, 1]
    x2 = table_16_15.iloc[row12, 1]
    y1 = table_16_15.iloc[row11, col]
    y2 = table_16_15.iloc[row12, col]
    Y1 = interpolation(x1, x2, y1, y2, v0)

    x1 = table_16_15.iloc[row21, 1]
    x2 = table_16_15.iloc[row22, 1]
    y1 = table_16_15.iloc[row21, col]
    y2 = table_16_15.iloc[row22, col]
    Y2 = interpolation(x1, x2, y1, y2, v0)

    x1 = table_16_15.iloc[row11, 0]
    x2 = table_16_15.iloc[row22, 0]
    fnpATS = round(interpolation(x1, x2, Y1, Y2, FFS), 2)
  elif col and row11 and row12:
    x1 = table_16_15.iloc[row11, 0]
    x2 = table_16_15.iloc[row12, 0]
    y1 = table_16_15.iloc[row11, col]
    y2 = table_16_15.iloc[row12, col]
    fnpATS = round(interpolation(x1, x2, y1, y2, FFS), 2)
  elif col and row21 and row22:
    x1 = table_16_15.iloc[row21, 1]
    x2 = table_16_15.iloc[row22, 1]
    y1 = table_16_15.iloc[row21, col]
    y2 = table_16_15.iloc[row22, col]
    fnpATS = round(interpolation(x1, x2, y1, y2, v0), 2)
  elif prevCol and nextCol and row:
    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row, prevCol]
    y2 = table_16_15.iloc[row, prevCol]
    fnpATS = round(interpolation(x1, x2, y1, y2, NPZ), 2)
  elif prevCol and nextCol and row11 and row12:
    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row11, prevCol]
    y2 = table_16_15.iloc[row11, nextCol]
    Y1 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row12, prevCol]
    y2 = table_16_15.iloc[row12, nextCol]
    Y2 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_15.iloc[row11, 0]
    x2 = table_16_15.iloc[row12, 0]
    fnpATS = round(interpolation(x1, x2, Y1, Y2, FFS), 2)
  elif prevCol and nextCol and row21 and row22:
    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row21, prevCol]
    y2 = table_16_15.iloc[row21, nextCol]
    Y1 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row22, prevCol]
    y2 = table_16_15.iloc[row22, nextCol]
    Y2 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_15.iloc[row21, 1]
    x2 = table_16_15.iloc[row22, 1]
    fnpATS = round(interpolation(x1, x2, Y1, Y2, v0), 2)
  else:
    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row11, prevCol]
    y2 = table_16_15.iloc[row11, nextCol]
    Y1 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row12, prevCol]
    y2 = table_16_15.iloc[row12, nextCol]
    Y2 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_15.iloc[row11, 0]
    x2 = table_16_15.iloc[row12, 0]
    Y1 = interpolation(x1, x2, Y1, Y2, FFS)

    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row21, prevCol]
    y2 = table_16_15.iloc[row21, nextCol]
    Y1 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_15.iloc[0, prevCol]
    x2 = table_16_15.iloc[0, nextCol]
    y1 = table_16_15.iloc[row22, prevCol]
    y2 = table_16_15.iloc[row22, nextCol]
    Y2 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_15.iloc[row21, 1]
    x2 = table_16_15.iloc[row22, 1]
    Y2 = interpolation(x1, x2, Y1, Y2, v0)

    x1 = table_16_15.iloc[row11, 0]
    x2 = table_16_15.iloc[row21, 0]
    fnpATS = round(interpolation(x1, x2, Y1, Y2, FFS), 2)

  prevId = 0
  col = None
  prevCol = None
  nextCol = None
  # pre calculation for fnp using table 16.16
  for id, item in enumerate(table_16_16.iloc[0]):
    if type(item) != str:
      if float(item) == float(NPZ):
        col = id
        break
      elif float(item) < float(NPZ):
        prevId = id
        if id == len(table_16_16.iloc[0]) - 1:
          col = id
      else:
        if id == 2:
          col = id
          break
        prevCol = prevId
        nextCol = id
        break

  row = None
  row11 = None
  row12 = None
  row21 = None
  row22 = None
  v0 = v1 + v2
  # fnp for PTSF using table 16.16
  for id, item in enumerate(table_16_16.iloc[:, 0]):
    if type(item) != str:
      vItem = table_16_16.iloc[:, 1][id]
      if float(item) == float(dir1):
        if float(vItem) == float(v0):
          row = id
          row11 = None
          row12 = None
          row21 = None
          row22 = None
          break
        elif float(vItem) < float(v0):
          if float(vItem) == 3200:
            row = id
            continue
          prevId = id
        else:
          if float(vItem) == 200:
            row = id
            continue
          row11 = None
          row12 = None
          row21 = prevId
          row22 = id
          break
      elif float(item) < float(dir1):
        if float(vItem) == float(v0):
          prevId = id
        elif float(vItem) < float(v0):
          if float(vItem) == 3200:
            row11 = id
            continue
          prevId = id
          row12 = None
        elif not row12:
          if float(vItem) == 200:
            row11 = id
            continue
          row11 = prevId
          row12 = id
      else:
        if float(vItem) == float(v0):
          row12 = id
          break
        elif float(vItem) < float(v0):
          if float(vItem) == 3200:
            row12 = id
            break
          prevId = id
        elif not row22:
          if float(vItem) == 200:
            row12 = id
            break
          row21 = prevId
          row22 = id
  if row11 and not row12:
    row = row11
    row11 = None
  elif row12 and not row11:
    row = row21
    row12 = None

  if col and row:
    fnpPTSF = table_16_16.iloc[row, col]
  elif col and row11 and row12 and row21 and row22:
    x1 = table_16_16.iloc[row11, 1]
    x2 = table_16_16.iloc[row12, 1]
    y1 = table_16_16.iloc[row11, col]
    y2 = table_16_16.iloc[row12, col]
    Y1 = interpolation(x1, x2, y1, y2, v0)

    x1 = table_16_16.iloc[row21, 1]
    x2 = table_16_16.iloc[row22, 1]
    y1 = table_16_16.iloc[row21, col]
    y2 = table_16_16.iloc[row22, col]
    Y2 = interpolation(x1, x2, y1, y2, v0)

    x1 = table_16_16.iloc[row11, 0]
    x2 = table_16_16.iloc[row22, 0]
    fnpPTSF = round(interpolation(x1, x2, Y1, Y2, dir1), 2)
  elif col and row11 and row12:
    x1 = table_16_16.iloc[row11, 0]
    x2 = table_16_16.iloc[row12, 0]
    y1 = table_16_16.iloc[row11, col]
    y2 = table_16_16.iloc[row12, col]
    fnpPTSF = round(interpolation(x1, x2, y1, y2, dir1), 2)
  elif col and row21 and row22:
    x1 = table_16_16.iloc[row21, 1]
    x2 = table_16_16.iloc[row22, 1]
    y1 = table_16_16.iloc[row21, col]
    y2 = table_16_16.iloc[row22, col]
    fnpPTSF = round(interpolation(x1, x2, y1, y2, v0), 2)
  elif prevCol and nextCol and row:
    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row, prevCol]
    y2 = table_16_16.iloc[row, prevCol]
    fnpPTSF = round(interpolation(x1, x2, y1, y2, NPZ), 2)
  elif prevCol and nextCol and row11 and row12:
    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row11, prevCol]
    y2 = table_16_16.iloc[row11, nextCol]
    Y1 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row12, prevCol]
    y2 = table_16_16.iloc[row12, nextCol]
    Y2 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_16.iloc[row11, 0]
    x2 = table_16_16.iloc[row12, 0]
    fnpPTSF = round(interpolation(x1, x2, Y1, Y2, dir1), 2)
  elif prevCol and nextCol and row21 and row22:
    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row21, prevCol]
    y2 = table_16_16.iloc[row21, nextCol]
    Y1 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row22, prevCol]
    y2 = table_16_16.iloc[row22, nextCol]
    Y2 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_16.iloc[row21, 1]
    x2 = table_16_16.iloc[row22, 1]
    fnpPTSF = round(interpolation(x1, x2, Y1, Y2, v0), 2)
  else:
    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row11, prevCol]
    y2 = table_16_16.iloc[row11, nextCol]
    Y1 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row12, prevCol]
    y2 = table_16_16.iloc[row12, nextCol]
    Y2 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_16.iloc[row11, 0]
    x2 = table_16_16.iloc[row12, 0]
    Y1 = interpolation(x1, x2, Y1, Y2, dir1)

    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row21, prevCol]
    y2 = table_16_16.iloc[row21, nextCol]
    Y1 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_16.iloc[0, prevCol]
    x2 = table_16_16.iloc[0, nextCol]
    y1 = table_16_16.iloc[row22, prevCol]
    y2 = table_16_16.iloc[row22, nextCol]
    Y2 = interpolation(x1, x2, y1, y2, NPZ)

    x1 = table_16_16.iloc[row21, 1]
    x2 = table_16_16.iloc[row22, 1]
    Y2 = interpolation(x1, x2, Y1, Y2, v0)

    x1 = table_16_16.iloc[row11, 0]
    x2 = table_16_16.iloc[row21, 0]
    fnpPTSF = round(interpolation(x1, x2, Y1, Y2, dir1), 2)

  return fnpATS, fnpPTSF

def calculateAB(v0):
  prevId = 0
  a = None
  b = None
  for id, item in enumerate(table_16_17["v"]):
    if float(item) == float(v0):
      a = table_16_17["a"][id]
      b = table_16_17["b"][id]
    elif float(item) < float(v0):
      prevId = id
    else:
      nextId = id
      x1 = table_16_17["v"][prevId]
      x2 = table_16_17["v"][nextId]

      y1 = table_16_17["a"][prevId]
      y2 = table_16_17["a"][nextId]
      a = round(interpolation(x1, x2, y1, y2, v0), 4)

      y1 = table_16_17["b"][prevId]
      y2 = table_16_17["b"][nextId]
      b = round(interpolation(x1, x2, y1, y2, v0), 4)
  return a, b

BFFS = input("Enter BFFS: ")
V = input("Enter Traffic Volume (vph): ")
PHF = input("Enter PHF: ")
dir1 = input("Enter direction 1 propertion (%): ")
dir2 = input("Enter direction 2 propertion (%): ")
PT = input("Enter Truck Percentage (%): ")
PR = input("Enter RV Percentage (%): ")
NPZ = input("Enter No Passing Zone Percentage (%): ")
highwayClass = input("Enter Highway Class (1/2): ")

v1 = round(float(V) * float(dir1) / 100 / float(PHF))
v2 = round(float(V) * float(dir2) / 100 / float(PHF))
print("v1: " + str(v1), "v2: " + str(v2))

print("Enter Grade Calculation Type:")
print("  1. Grade Type")
print("  2. Grade Value")
gradeCalculationType = input()

gradeType = None
grade = None
gradeLength = None
if gradeCalculationType == "1":
  print("Enter Grade Type:")
  print("  1. Level Terrain")
  print("  2. Rolling Terrain")
  gradeType = input()
else:
  grade = input("Grade (%): ")
  gradeLength = input("Grade Length (mi): ")

fGATS1, fGPTSF1 = calculateFG(gradeCalculationType, v1, gradeType, grade, gradeLength)
fGATS2, fGPTSF2 = calculateFG(gradeCalculationType, v2, gradeType, grade, gradeLength)

fHVATS1, fHVPTSF1 = calculateFHV(gradeCalculationType, v1, PT, PR, gradeType, grade, gradeLength)
fHVATS2, fHVPTSF2 = calculateFHV(gradeCalculationType, v2, PT, PR, gradeType, grade, gradeLength)

print("Enter FFS Calculation Type:")
print("  1. Using Sm")
print("  2. Using Table")
FFSCalculationType = input()

Sm = None
vf = None
laneWidth = None
shoulderWidth = None
accessPoints = None

if FFSCalculationType == "1":
  Sm = input("Enter Sm: ")
  FFS = Sm + 0.00776 * float(V) / min(fHVATS1, fHVPTSF1, fHVATS2, fHVPTSF2)
else:
  laneWidth = input("Enter Lane Width: ")
  shoulderWidth = input("Enter Shoulder Width: ")
  accessPoints = input("Enter Number of Access Points per Mile: ")

  fLS = calculateFLS(laneWidth, shoulderWidth)
  print("fLS: " + str(fLS))

  fA = calculateFA(accessPoints)
  print("fA: " + str(fA))

  FFS = float(BFFS) - fLS - fA
  print("FFS: " + str(FFS))

vATS1 = round(float(V) * float(dir1) / 100 / (float(PHF) * fHVATS1 * fGATS1))
vPTSF1 = round(float(V) * float(dir1) / 100 / (float(PHF) * fHVPTSF1 * fGPTSF1))
vATS2 = round(float(V) * float(dir2) / 100 / (float(PHF) * fHVATS2 * fGATS2))
vPTSF2 = round(float(V) * float(dir2) / 100 / (float(PHF) * fHVPTSF2 * fGPTSF2))

fnpATS1, fnpPTSF1 = calculatefnp(FFS, v2, NPZ, max(float(dir1), float(dir2)), v1, v2)
fnpATS2, fnpPTSF2 = calculatefnp(FFS, v1, NPZ, max(float(dir1), float(dir2)), v1, v2)

ATS1 = round(FFS - 0.00776 * (v1 + v2) - fnpATS1, 2)
ATS2 = round(FFS - 0.00776 * (v1 + v2) - fnpATS2, 2)

a1, b1 = calculateAB(v2)
a2, b2 = calculateAB(v1)

BPTSF1 = 100 * (1 - math.exp(a1 * pow(v1, b1)))
BPTSF2 = 100 * (1 - math.exp(a2 * pow(v2, b2)))
PTSF1 = round(BPTSF1 + fnpPTSF1 * v1 / (v1 + v2), 2)
PTSF2 = round(BPTSF2 + fnpPTSF2 * v2 / (v1 + v2), 2)

if highwayClass == "1":
  print("fG1(ATS): " + str(fGATS1), "fG2(ATS): " + str(fGATS2))
  print("fG1(PTSF): " + str(fGPTSF1), "fG2(PTSF): " + str(fGPTSF2))
  print("fHV1(ATS): " + str(fHVATS1), "fHV2(ATS): " + str(fHVATS2))
  print("fHV1(PTSF): " + str(fHVPTSF1), "fHV2(PTSF): " + str(fHVPTSF2))
  print("v1(ATS): " + str(vATS1), "v2(ATS): " + str(vATS2))
  print("v1(PTSF): " + str(vPTSF1), "v2(PTSF): " + str(vPTSF2))
  print("fnp1(ATS): " + str(fnpATS1), "fnp2(ATS): " + str(fnpATS2))
  print("fnp1(PTSF): " + str(fnpPTSF1), "fnp2(PTSF): " + str(fnpPTSF2))
  print("a1(PTSF): " + str(a1), "b1(PTSF): " + str(b1))
  print("a2(PTSF): " + str(a2), "b2(PTSF): " + str(b2))
  print("ATS1: " + str(ATS1), "ATS2: " + str(ATS2))
  print("PTSF1: " + str(PTSF1) + "%", "PTSF2: " + str(PTSF2) + "%")

  row = None
  for id, item in enumerate(table_16_4["ats_class1"]):
    if type(item) == str:
      range = item.split(",")
      if len(range) == 2:
        rangeFrom = range[0]
        rangeUpto = range[1]
        if rangeFrom and rangeUpto:
          if float(rangeFrom) <= float(ATS1) < float(rangeUpto):
            row = id
        elif rangeFrom:
          if float(rangeFrom) <= float(ATS1):
            row = id
        else:
          if float(rangeUpto) > float(ATS1):
            row = id
  print("ATS1 Level of Service: " + table_16_4["level_of_service"][row])

  for id, item in enumerate(table_16_4["ptsf_class1"]):
    if type(item) == str:
      range = item.split(",")
      if len(range) == 2:
        rangeFrom = range[0]
        rangeUpto = range[1]
        if rangeFrom and rangeUpto:
          if float(rangeFrom) <= float(PTSF1) < float(rangeUpto):
            row = id
        elif rangeFrom:
          if float(rangeFrom) <= float(PTSF1):
            row = id
        else:
          if float(rangeUpto) > float(PTSF1):
            row = id
  print("PTSF1 Level of Service: " + table_16_4["level_of_service"][row])

  for id, item in enumerate(table_16_4["ats_class1"]):
    if type(item) == str:
      range = item.split(",")
      if len(range) == 2:
        rangeFrom = range[0]
        rangeUpto = range[1]
        if rangeFrom and rangeUpto:
          if float(rangeFrom) <= float(ATS2) < float(rangeUpto):
            row = id
        elif rangeFrom:
          if float(rangeFrom) <= float(ATS2):
            row = id
        else:
          if float(rangeUpto) > float(ATS2):
            row = id
  print("ATS2 Level of Service: " + table_16_4["level_of_service"][row])

  for id, item in enumerate(table_16_4["ptsf_class1"]):
    if type(item) == str:
      range = item.split(",")
      if len(range) == 2:
        rangeFrom = range[0]
        rangeUpto = range[1]
        if rangeFrom and rangeUpto:
          if float(rangeFrom) <= float(PTSF2) < float(rangeUpto):
            row = id
        elif rangeFrom:
          if float(rangeFrom) <= float(PTSF2):
            row = id
        else:
          if float(rangeUpto) > float(PTSF2):
            row = id
  print("PTSF2 Level of Service: " + table_16_4["level_of_service"][row])
else:
  print("fG1(PTSF): " + str(fGPTSF1), "fG2(PTSF): " + str(fGPTSF2))
  print("fHV1(PTSF): " + str(fHVPTSF1), "fHV2(PTSF): " + str(fHVPTSF2))
  print("v1(PTSF): " + str(vPTSF1), "v2(PTSF): " + str(vPTSF2))
  print("fnp1(PTSF): " + str(fnpPTSF1), "fnp2(PTSF): " + str(fnpPTSF2))
  print("a1(PTSF): " + str(a1), "b1(PTSF): " + str(b1))
  print("a2(PTSF): " + str(a2), "b2(PTSF): " + str(b2))
  print("PTSF1: " + str(PTSF1) + "%", "PTSF2: " + str(PTSF2) + "%")

  row = None
  for id, item in enumerate(table_16_4["ptsf_class2"]):
    if type(item) == str:
      range = item.split(",")
      if len(range) == 2:
        rangeFrom = range[0]
        rangeUpto = range[1]
        if rangeFrom and rangeUpto:
          if float(rangeFrom) <= float(PTSF1) < float(rangeUpto):
            row = id
        elif rangeFrom:
          if float(rangeFrom) <= float(PTSF1):
            row = id
        else:
          if float(rangeUpto) > float(PTSF1):
            row = id
  print("PTSF1 Level of Service: " + table_16_4["level_of_service"][row])

  for id, item in enumerate(table_16_4["ptsf_class2"]):
    if type(item) == str:
      range = item.split(",")
      if len(range) == 2:
        rangeFrom = range[0]
        rangeUpto = range[1]
        if rangeFrom and rangeUpto:
          if float(rangeFrom) <= float(PTSF2) < float(rangeUpto):
            row = id
        elif rangeFrom:
          if float(rangeFrom) <= float(PTSF2):
            row = id
        else:
          if float(rangeUpto) > float(PTSF2):
            row = id
  print("PTSF2 Level of Service: " + table_16_4["level_of_service"][row])

Enter BFFS: 60
Enter Traffic Volume (vph): 1000
Enter PHF: 1
Enter direction 1 propertion (%): 60
Enter direction 2 propertion (%): 40
Enter Truck Percentage (%): 2
Enter RV Percentage (%): 1
Enter No Passing Zone Percentage (%): 3
Enter Highway Class (1/2): 1
v1: 600 v2: 400
Enter Grade Calculation Type:
  1. Grade Type
  2. Grade Value
1
Enter Grade Type:
  1. Level Terrain
  2. Rolling Terrain
1
Enter FFS Calculation Type:
  1. Using Sm
  2. Using Table
2
Enter Lane Width: 12
Enter Shoulder Width: 8
Enter Number of Access Points per Mile: 1
fLS: 0
fA: 0.25
FFS: 59.75
fG1(ATS): 1 fG2(ATS): 1
fG1(PTSF): 1 fG2(PTSF): 1
fHV1(ATS): 0.998 fHV2(ATS): 0.994
fHV1(PTSF): 1.0 fHV2(PTSF): 0.998
v1(ATS): 601 v2(ATS): 402
v1(PTSF): 600 v2(PTSF): 401
fnp1(ATS): 1.4 fnp2(ATS): 1.09
fnp1(PTSF): 14.81 fnp2(PTSF): 14.81
a1(PTSF): -0.0022 b1(PTSF): 0.9434
a2(PTSF): -0.003 b2(PTSF): 0.8985
ATS1: 50.59 ATS2: 50.9
PTSF1: 69.0% PTSF2: 53.89%
ATS1 Level of Service: B
PTSF1 Level of Service: D
ATS2 Level of 